In [7]:
import os
from pcg_gazebo.generators import ModelGroupGenerator

generator = ModelGroupGenerator('full_crate')

In [8]:
generator.add_asset(
    tag='crate',
    description=dict(
        type='mesh',
        args=dict(
            visual_mesh='file://' + os.path.abspath('meshes/crate.stl'),
            visual_mesh_scale=[1, 1, 1],
            use_approximated_collision=False,
            name='crate',
            color='xkcd'
        )
    )
)

True

In [9]:
generator.add_asset(
    tag='crate_ball',
    description=dict(
        type='sphere',
        args=dict(
            radius="max(0.05, 0.1 * __import__('numpy').random.random())",
            name='sphere',
            mass="max(0.1, __import__('numpy').random.random())",
            color='xkcd'
        )
    )
)

generator.add_asset(
    tag='crate_cuboid',
    description=dict(
        type='box',
        args=dict(
            size="0.1 * __import__('numpy').random.random(3)",
            name='cuboid',
            mass="max(0.01, __import__('numpy').random.random())",
            color='xkcd'
        )
    )
)

generator.add_asset(
    tag='crate_cylinder',
    description=dict(
        type='cylinder',
        args=dict(
            length="max(0.05, 0.1 *__import__('numpy').random.random())",
            radius="max(0.05, 0.1 *__import__('numpy').random.random())",
            name='cuboid',
            mass="max(0.01, __import__('numpy').random.random())",
            color='xkcd'
        )
    )
)

crate_assets = ['crate_ball', 'crate_cuboid', 'crate_cylinder']

In [10]:
generator.add_constraint(
    name='tangent_to_ground_plane',
    type='tangent',
    frame='world',
    reference=dict(
        type='plane',
        args=dict(
            origin=[0, 0, 0],
            normal=[0, 0, 1]
        )
    )
)

True

In [12]:
generator.add_constraint(
    name='crate_base',
    type='workspace',
    frame='world',
    geometry=dict( 
        type='area',
        description=dict(
            points=[ 
                [-0.5, -0.4, 0],
                [-0.5, 0.4, 0],
                [0.5, 0.4, 0],
                [0.5, -0.4, 0]
            ]
        )
    )
)

NotImplementedError: Invalid geometry type, provided=None

In [13]:
generator.add_engine(
    tag='crate_engine',
    engine_name='fixed_pose',
    models=['crate'],
    poses=[
        [0, 0, 0, 0, 0, 0]
    ],
    constraints=[
        dict(
            model='crate',
            constraint='tangent_to_ground_plane'
        )
    ]
)

True

In [14]:
num_instances = dict()
for tag in crate_assets:
    num_instances[tag] = 4

generator.add_engine(
    tag='fill_crate',
    engine_name='random_pose',
    models=crate_assets,
    max_num=num_instances,
    model_picker='random',
    no_collision=True,
    policies=[
        dict(
            models=crate_assets,
            config=[
                dict(
                    dofs=['x', 'y'],
                    tag='workspace',
                    workspace='crate_base'
                ),
                dict(
                    dofs=['z'],
                    tag='uniform',
                    mean=0.5,
                    min=0.0,
                    max=3.0
                ),
                dict(
                    dofs=['roll', 'pitch', 'yaw'],
                    tag='uniform',
                    mean=0,
                    min=-3.141592653589793,
                    max=3.141592653589793
                )
            ]
        )
    ]
)

True

In [15]:
models = list()
for i in range(3):
    models.append(generator.run(group_name='full_crate_{}'.format(i)))
    print('Full crate model #{} generated, # nested models={}'.format(i, models[-1].n_models))

2021-01-01 02:45:50,743 | WARNING | path | File /root/pcg_tameshi/meshes/crate.stl does not exist


AssertionError: Filename could not be resolved from file:///root/pcg_tameshi/meshes/crate.stl